In [1]:
import os
os.chdir('/home/xinglong/git_local/dynamax/')

import matplotlib.pyplot as plt

In [2]:
import jax.numpy as jnp
import jax.random as jr
from jax import lax

from dynamax.structural_time_series.models.sts_model import StructuralTimeSeries as STS
from dynamax.structural_time_series.models.sts_components import *
from dynamax.parameters import ParameterProperties as Prop

import tensorflow_probability as tfp
from tensorflow_probability.substrates.jax.distributions import (
    MultivariateNormalDiag as MVNDiag
)


In [3]:
time_steps=100
key=jr.PRNGKey(4)

keys = jr.split(key, 5)
standard_mvn = MVNDiag(jnp.zeros(1), jnp.ones(1))

# Generate parameters of the STS component
level_scale = 5
slope_scale = 0.5
initial_level = standard_mvn.sample(seed=keys[0])
initial_slope = standard_mvn.sample(seed=keys[1])

obs_noise_scale = 20

# Generate observed time series using the SSM representation.
F = jnp.array([[1, 1],
               [0, 1]])
H = jnp.array([[1, 0]])
Q = jnp.block([[level_scale, 0],
                   [0, slope_scale]])
R = obs_noise_scale

def _step(current_state, key):
    key1, key2 = jr.split(key)
    current_obs = H @ current_state + R * standard_mvn.sample(seed=key1)
    next_state = F @ current_state + Q @ MVNDiag(jnp.zeros(2), jnp.ones(2)).sample(seed=key2)
    return next_state, current_obs

initial_state = jnp.concatenate((initial_level, initial_slope))
key_seq = jr.split(keys[2], time_steps)
_, obs_time_series = lax.scan(_step, initial_state, key_seq)

In [4]:
tfp_comp = tfp.sts.LocalLinearTrend(observed_time_series=obs_time_series)
tfp_model = tfp.sts.Sum([tfp_comp], observed_time_series=obs_time_series)

2022-11-02 23:27:04.315471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-11-02 23:27:04.315494: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
tfp_samples, _ = tfp.sts.fit_with_hmc(
        model=tfp_model, observed_time_series=obs_time_series, seed=keys[3])


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
Do not call `graph_parents`.


Instructions for updating:
Do not call `graph_parents`.
/home/xinglong/.local/lib/python3.8/site-packages/tensorflow_probability/python/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '


In [17]:
tfp_model.parameters[2]

Parameter(name='LocalLinearTrend/_slope_scale', prior=<tfp.distributions.LogNormal 'slope_scale_prior' batch_shape=[] event_shape=[] dtype=float32>, bijector=<tfp.bijectors.Chain 'chain_of_scale_of_softplus' batch_shape=[] forward_min_event_ndims=0 inverse_min_event_ndims=0 dtype_x=float32 dtype_y=float32 bijectors=[<tfp.bijectors.Scale 'scale' batch_shape=[] forward_min_event_ndims=0 inverse_min_event_ndims=0 dtype_x=float32 dtype_y=float32>, <tfp.bijectors.Softplus 'softplus' batch_shape=[] forward_min_event_ndims=0 inverse_min_event_ndims=0 dtype_x=float32 dtype_y=float32>]>)